# Twitter API


### Import Dependencies

In [1]:
from config import (consumer_key, consumer_secret, 
                    access_token, access_token_secret)

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tweepy
import json
import datetime
import time
%matplotlib inline

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

## Setup Tweepy API Authentication

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [17]:
target_user = ["@BBC","@CBS","@CNN","@FoxNews","@nytimes"]
sentiments = []

## Create a module for analyzing twitter sentiments

In [25]:
def twitter_search(user):
    for x in range(1, 6):

        # Get all tweets from home feed
        public_tweets = api.user_timeline(user, page=x)
    
        # Loop through all tweets
        for tweet in public_tweets:
    
            # Run Vader Analysis on each tweet
            results = analyzer.polarity_scores(tweet["text"])
            compound = results["compound"]
            pos = results["pos"]
            neu = results["neu"]
            neg = results["neg"]
    
            # Add each value to the appropriate list
            compound_list.append(compound)
            positive_list.append(pos)
            negative_list.append(neg)
            neutral_list.append(neu)
            
            sentiments.append({"Date": tweet["created_at"], 
                               "Compound": compound,
                               "Positive": pos,
                               "Negative": neu,
                               "Neutral": neg,
                               "Text": tweet['text']})
            
            time = public_tweets[0]['created_at']
            convert_time(time)
            
    name = user.strip( '@' )
    file_name = 'Output/' + name + '_compound'
    plt.plot(compound_list, marker = 'o')
    plt.title(f"Sentiment Analysis of Tweets for {user} {time}")
    plt.ylabel("Tweet Polarity")
    plt.xlabel("Time stamp")
    plt.xticks(np.arange(len(compound_list), step = 10), rotation=90)
    #plt.tight_layout()
    plt.savefig(file_name)
    plt.clf()

## Create a module to converting time

In [6]:
def convert_time(time):
    time = datetime.datetime.strptime(time, "%a %b %d %H:%M:%S %z %Y").timestamp()
    time = datetime.datetime.fromtimestamp(time).strftime('%Y-%m-%d %H:%M:%S')
    time_stamp.append(time)

## Run loop to create plots

In [26]:
for user in target_user:
    # Variables for holding sentiments
    compound_list = []
    positive_list = []
    negative_list = []
    neutral_list = []
    time_stamp = []
    twitter_search(user)

<Figure size 432x288 with 0 Axes>

In [27]:
# Convert sentiments to DataFrame
sentiments_pd = pd.DataFrame.from_dict(sentiments)
sentiments_pd.head()

,Compound,Date,Negative,Neutral,Positive,Text
0,0.6696,Sun Jul 08 08:02:03 +0000 2018,0.572,0.000,0.428,😍🏝 Paradise is on our doorstep! https://t.co/...
1,0.0000,Sat Jul 07 19:09:04 +0000 2018,1.000,0.000,0.000,'You don't look autistic' and other things not...
2,0.3818,Sat Jul 07 18:05:03 +0000 2018,0.852,0.000,0.148,A South African woman is recovering in hospita...
3,-0.6908,Sat Jul 07 17:05:04 +0000 2018,0.612,0.388,0.000,The women paid to cry at the funerals of stran...
4,0.0000,Sat Jul 07 16:03:07 +0000 2018,1.000,0.000,0.000,These are the nine TV shows you NEED to watch ...


In [24]:
sentiments_pd.to_csv('Output/sentiments_pd.csv')

## Analysis of Plots
- There does not seem to be any correlations between time and sentiments of the tweets from each news source.
- CBS tends to be slightly more positive than the others, but most of them are pretty scattered from time to time. 
- The news from each media source is very polarized and ranges from the most negative to most positive.